# Парсер 2ГИС

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from tqdm import tqdm

# Функция для парсинга типа здания с 2GIS
def get_building_type(address, driver):
    try:
        # Открываем сайт 2GIS
        driver.get('https://2gis.ru/')

        # Ждем, пока поле ввода станет доступным
        wait = WebDriverWait(driver, 10)
        input_element = wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "_h7eic2"))
        )

        # Вводим адрес
        input_element.clear()
        input_element.send_keys(address)
        input_element.send_keys(Keys.RETURN)

        # Ждем загрузки результатов поиска
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "_5b28jpo")))

        # Получаем тип здания
        try:
            building_type_element = driver.find_element(By.CSS_SELECTOR, "div._1idnaau span._oqoid")
            return building_type_element.text
        except NoSuchElementException:
            return "Не удалось определить тип здания"
    except TimeoutException:
        return "Ошибка загрузки"

In [ ]:
df = pd.read_excel("Результат.xlsx", dtype="str", sheet_name = "Сводная")   
df['Тип_здания_2ГИС'] = ""

In [ ]:
# Основной скрипт
if __name__ == "__main__":
    # Пример DataFrame с адресами
    # Настраиваем драйвер Selenium
    driver = webdriver.Firefox()
    try:
        for index, row in tqdm(df.iterrows()):
            df.at[index, 'Тип_здания_2ГИС'] = get_building_type(str(row['Исходный_адрес']), driver)
    finally:
        # Закрываем браузер
        driver.quit()
    # Сохраняем результат в файл или выводим на экран

In [ ]:
df.to_excel("Результат_с_2ГИС.xlsx", index=False)